# Codetober. II: Scientific Computing in Python
Session leader: Matthew Lundy (matthew.lundy@mail.mcgill.ca)

Notebook Author: Jordan Mirocha (jordan.mirocha@mcgill.ca)

**Welcome!** 

Before we start, be sure to download the data we'll use today:

https://www.kaggle.com/aubertsigouin/biximtl/download

This should download a file called `archive.zip`. Once finished, I'd recommend making a new folder for today's session, e.g.,:

```text
cd ~/Desktop
mkdir codetober_2
cd codetober_2
mv ~/Downloads/archive.zip .
unzip archive.zip
jupyter notebook
```

#### it will also be convenient if you copy this notebook into that new folder. 

## So...what is *scientific* computing?

When people emphasize that something is *scientific* computing, as opposed to just *computing* computing, they generally mean that the computations in question allow us to better understand natural systems through data analysis and modeling techniques that would be impossible to execute by hand. The data here is assumed to be numerical, and we operate on this data (or create it) with algorithms whose fundamental language is mathematics.

In other words, it's computing that helps us do science better.

### Scope

This is a big topic -- we'll only scratch the surface in the next 1.5 hours. With that in mind, **the goal is to touch on the tools and techniques you're most likely to need when you first get involved with research**, to help ease that transition. But, I'll also sprinkle in some tricks of the trade that will be useful as you start to build more sophisticated software, so if you already have some research experience, this workshop should still be worth your while.

No prior experience with Python or scientific computing is necessary!

### Plan for today 

There are many fantastic resources online for learning Python, some of which I'll draw upon here (e.g., [arrays](https://github.com/profjsb/python-bootcamp/blob/master/Lectures/21_NumpyMatplotlib/IntroNumPy.ipynb), [plotting](https://github.com/profjsb/python-bootcamp/blob/master/Lectures/21_NumpyMatplotlib/IntroMatplolib.ipynb)), scientific computing techniques, etc.

BUT, my biggest complaint about a lot of online tutorials is that there is not very much context. It's a lot of trying to learn/memorize Python syntax without any reasoning for why knowing these things is good.

So, today we're going to get our hands dirty with some publicly available [BIXI data](https://www.kaggle.com/aubertsigouin/biximtl). Just by "playing with" this data and asking simple questions about it, we will learn:

- How to read and write basic data files (very common)
- How to find interesting parts of datasets that are too big to sift through entirely by eye.
- How to slice and dice the data to do interesting analyses.
- How to plot data, including time series, images, and histograms.
- How to do some simple model fitting.

Here's [an example](https://towardsdatascience.com/understanding-bixi-commuters-an-analysis-of-montreals-bike-share-system-in-python-cb34de0e2304) of what other people are doing with this data.

## Disclaimer! I don't know anything about transportation.

## Getting started: imports and key packages

Your new best friends: `numpy` and `matplotlib`. We'll talk about `scipy` later too.

In [ ]:
%pylab inline
import numpy as np
import matplotlib.pyplot as pl

`numpy` is short for "numerical Python". It allows us to create and manipulate *arrays*, which are like lists or tuples, but are designed to contain a single data type (generally integers or floats). Unlike lists, we can perform mathematical operations on arrays, e.g.,

In [ ]:
my_list = [1,2,3,4]
my_array = np.arange(1, 5)
print(my_list * 2, my_array * 2)

Multiplying lists by an integer $N$ creates a new list containing the same elements repeated $N$ times! 

Multiplying an array by a number $N$ yields the more intuitive result, in which each entry is multiplied by $N$. In other words, the operation is performed "element-wise," i.e., element by element.

You can reproduce what `numpy` does using `for` loops:

In [ ]:
new_list = []
for element in my_list:
    new_list.append(2 * element)
    
print(new_list)

However, this is *a lot* slower than the equivalent operation in `numpy`, especially for really big arrays. This is because `numpy` is really a wrapper around compiled C code.

**In scientific computing, avoid using loops at all costs!**

## Reading in the BIXI data

Hopefully, anytime you download a dataset there's some README file that tells you how the data is structured. However, in this case, the data is a human-readable format, so we can just glance at it to see what's in there. If you open it in a text editor, or look at it on the command line via `less`, it should look something like:

```text
,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2014-04-15 00:01,6209,2014-04-15 00:18,6436,1061,1
1,2014-04-15 00:01,6214,2014-04-15 00:11,6248,615,1
2,2014-04-15 00:01,6164,2014-04-15 00:18,6216,1031,1
3,2014-04-15 00:01,6214,2014-04-15 00:24,6082,1382,1
4,2014-04-15 00:02,6149,2014-04-15 00:08,6265,347,1
5,2014-04-15 00:05,6214,2014-04-15 00:08,6211,167,1
```

This is for the file `OD_2014.csv`. There are separate files for the station info, e.g., in `Stations_2014.csv` we have:

```text
code,name,latitude,longitude
6209,Milton / Clark,45.51252,-73.57062
6436,Côte St-Antoine / Clarke,45.48645209646392,-73.59523415565491
6214,Square St-Louis,45.51735,-73.56906
6248,St-Dominique / Rachel,45.518593,-73.581566
6164,Chambord / Laurier,45.5329550401632,-73.58419418334961
6216,Parc Jeanne Mance (monument George-Étienne Cartier),45.51496,-73.58503
```

`numpy` has a convenience routine called `loadtxt` to help read in tabular data like this. Let's give that a try:

In [ ]:
filepath= 'C:/Users/matth/Desktop/codetober_2/' #change based on your username/file location. If your notebook is in the same folder
#you can set this to ''
data_2014 = np.loadtxt(filepath+'OD_2014.csv', delimiter=',', usecols=[0,5], skiprows=1)

#### Q. How many BIXI rides in 2014?
Can determine just from the shape of the dataset:

In [ ]:
print(np.shape(data_2014))
print(data_2014.shape)

#### Q. Has ridership increased over time?

In [ ]:
data_2017 = np.loadtxt(filepath+'OD_2017.csv', delimiter=',', usecols=[0,5], skiprows=1)

In [ ]:
print(data_2017.shape[0] / float(data_2014.shape[0]))

#### Q. Did the number of stations change?

In [ ]:
stat_2014 = np.loadtxt(filepath+'Stations_2014.csv', delimiter=',', usecols=[0], skiprows=1)
stat_2017 = np.loadtxt(filepath+'Stations_2017.csv', delimiter=',', usecols=[0], skiprows=1)
print(stat_2017.shape[0] / float(stat_2014.shape[0]))

#### How long was the typical ride in 2014?

In [ ]:
# Convert ride times to minutes
duration_min_2014 = data_2014[:,1] / 60.

# Compute mean
mean_ride_2014 = np.mean(duration_min_2014)

#Compute median

median_ride_2014 = np.median(duration_min_2014)

# Compute the mode
hist, bin_edges = np.histogram(duration_min_2014, bins=100)
mode_ride_2014 = bin_edges[hist.argmax()] 
# Note: this is a little sloppy: should take position of mode to be at bin *center*


print(f'Mean ride = {mean_ride_2014:.2f}')
print(f'Median ride = {median_ride_2014:.2f}')
print(f'Mode of ride durations = {mode_ride_2014:.2f}')


#### How big of a spread is there in ride times?

In [ ]:
plt.figure(figsize=(15,5))
hist = pl.hist(duration_min_2014, bins=np.arange(1, 101), color='b', alpha=0.5)
pl.axvline(mean_ride_2014, color='k', ls='--',label='Mean')
pl.axvline(median_ride_2014, color='k', ls='dashdot',label='Median')
pl.axvline(mode_ride_2014, color='k', ls=':',label='Mode')
plt.legend()

pl.xlabel('Ride duration [min]')
pl.ylabel('Number of rides in 2014')

In [ ]:
plt.figure(figsize=(15,5))
hist = pl.hist(duration_min_2014, bins=np.arange(1, 101), color='b', alpha=0.5)
pl.axvline(mean_ride_2014, color='k', ls='--',label='Mean')
pl.axvline(median_ride_2014, color='k', ls='dashdot',label='Median')
pl.axvline(mode_ride_2014, color='k', ls=':',label='Mode')
plt.legend()
plt.yscale('log')
pl.xlabel('Ride duration [min]')
pl.ylabel('Number of rides in 2014')

## How to deal with dates in the dataset?

One tricky thing about this dataset is that there is a mix of data types, which is going to cause problems for `numpy`, since each array needs to contain only variables of the same type!

This is a great reason to use the `pandas` package, which has much more flexible file-reading machinery available.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(filepath+'OD_2014.csv')
stations = pd.read_csv(filepath+'Stations_2014.csv')
data.head()

In [ ]:
stations.head()

Note that `data` and `stations` are now *objects*. Rather than slicing a pure `numpy` array, we access each column by name. To see what columns are available, we can do:

In [ ]:
print(data.columns)
print(stations.columns)

In [ ]:
import datetime

In [ ]:
s = data['start_date'][0]
date = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M")

In [ ]:
jan1 = datetime.datetime(date.year, 1, 1, 0, 0)
jan10 = datetime.datetime(date.year, 1, 10, 0, 0)
diff = (jan10 - jan1)

In [ ]:
diff.total_seconds()

In [ ]:
date.__getattribute__('second')

In [ ]:
cols = 'month', 'day', 'hour'
start = {col:[] for col in cols}
stop  = {col:[] for col in cols}
duration = np.array([])
subsamp=20
for i in range(int(data.shape[0]/20)):
    _start = datetime.datetime.strptime(data['start_date'][i], "%Y-%m-%d %H:%M")
    _stop = datetime.datetime.strptime(data['end_date'][i], "%Y-%m-%d %H:%M")
    
    for col in cols:
        start[col].append(_start.__getattribute__(col))
        stop[col].append(_stop.__getattribute__(col))

    diff = _stop - _start
    duration=np.append(duration,diff.total_seconds()/60.)

#### Q. When do people ride most throughout the day?

In [ ]:
plt.figure(figsize=(15,7))
h = pl.hist(start['hour'], bins=np.arange(0, 25),edgecolor='black', linewidth=1.2)
plt.xlabel('Time of Day (hour)')
plt.ylabel('Number of trips')
plt.xlim(0,24)

#### Q. Are commutes longer than non-commutes?

In [ ]:
hours = np.arange(0, 24)
oksum=0
binconts=np.zeros(24)
for i,hour in enumerate(hours):
    ok = np.logical_and(start['hour'] >= hour, start['hour'] < hour+1)
    binconts[i]=np.sum(duration[ok])/ok.sum()
pl.figure(figsize=(15,7))
plt.axvspan(6,9,color='darkred',alpha=0.5)
plt.axvspan(15,18,color='darkred',alpha=0.5)
pl.bar(hours+0.5,binconts,width=1,edgecolor='k',fill=False)
pl.xlabel('Time of Day (hour)')
plt.ylabel('Average ride duration')
plt.xlim(0,24)
plt.ylim(ymin=10)
index=[6,7,8,15,16,17]
print(f'Average time during rush hour : {np.mean(binconts[index]):.2f}')
print(f'Average time outside of rush hour : {np.mean(binconts[[i for i in range(len(binconts)) if i not in index]]):.2f}')

#### Q. What about weekday vs. weekend?

In [ ]:
day = np.array(start['day'])
is_weekend = np.logical_and(day >= 5, day <= 6)

fig = pl.figure(figsize=(11,8))
frame1=fig.add_axes((.1,.3,.8,.6))
h1=pl.hist(duration_min_2014[:int(data.shape[0]/20)][is_weekend==1], color='darkblue', density=True, bins=np.arange(1, 100),alpha=0.5,label="Weekend")
h2=pl.hist(duration_min_2014[:int(data.shape[0]/20)][is_weekend==0], color='darkred', density=True, bins=np.arange(1, 100),alpha=0.5,label="Weekday")
frame1.set_xticklabels([]) 
pl.grid()
pl.ylabel('Density of Bike Trips')


frame2=fig.add_axes((.1,.1,.8,.2))        
pl.bar(h1[1][:-1],h1[0]-h2[0],width=1,color='k')
grid()
pl.ylabel('Difference')
pl.xlabel('Duration (min)')


#### Q. Where are BIXIs piling up? And where are they running out?

In [ ]:
landmarks = \
{
 'Parc La Fontaine': (-73.570392, 45.527527),
 'McGill': (-73.57722458691691, 45.504616238406314),  
 'UdeM': (-73.61220069267776, 45.50681365964249),
 'Mont Royal Chalet': (-73.58755377496506, 45.503930433540475)
}

In [ ]:
pl.figure(figsize=(10,8))
pl.scatter(stations['longitude'], stations['latitude'], s=10)

for landmark in landmarks:
    lon, lat = landmarks[landmark]
    pl.scatter(lon, lat, marker='s', s=40, label=landmark)
pl.grid()
pl.ylabel('Lat (deg)')
pl.xlabel('Lon (deg)')
pl.legend(loc='lower right')

In [ ]:
pl.figure(figsize=(10,8))
hist, be1, be2, patches = pl.hist2d(stations['longitude'], stations['latitude'], bins=20)
cbar=pl.colorbar()
cbar.set_label('Density of Stations', rotation=270)

pl.ylabel('Lat (deg)')
pl.xlabel('Lon (deg)')

In [ ]:
datetest=np.arange(15,20)
for d in datetest:
    x=[]
    y=[]
    z=[]
    for index, row in data.iterrows():
        if (start['day'][index]==int(d)) and (start['month'][index]==4):
            x.append(stations.loc[stations['code'] == row['start_station_code']]['longitude'].values[0])
            y.append(stations.loc[stations['code'] == row['start_station_code']]['latitude'].values[0])
            z.append(1)
        
            x.append(stations.loc[stations['code'] == row['end_station_code']]['longitude'].values[0])
            y.append(stations.loc[stations['code'] == row['end_station_code']]['latitude'].values[0])
            z.append(-1)
        
        if index > 100000:
            break
    pl.figure(figsize=(10,8))
    pl.title(f'Date of {d}th of April 2014')
    hist, be1, be2, patches = pl.hist2d(x,y,weights=z, bins=30,cmap='RdGy_r',vmin=-50,vmax=50)
    
    cbar=pl.colorbar()
    cbar.set_label('Difference in Bikes', rotation=270)
    pl.ylabel('Lat (deg)')
    pl.xlabel('Lon (deg)')
    for landmark in landmarks:
        lon, lat = landmarks[landmark]
        pl.scatter(lon, lat, marker='s', s=40, label=landmark)
    pl.legend(loc='lower right')
    plt.show()

#### What are the long term trends in the most active station like?

In [ ]:
popstat=data.start_station_code.mode()[0]

In [ ]:
x=np.array([0])
y=np.array([0])
for index, row in data.iterrows():
    if row['start_station_code']==popstat:
        x=np.append(x,datetime.datetime.strptime(data['start_date'][index], "%Y-%m-%d %H:%M"))
        y=np.append(y,y[-1]+1)
    if row['end_station_code']==popstat:
        x=np.append(x,datetime.datetime.strptime(data['end_date'][index], "%Y-%m-%d %H:%M"))
        y=np.append(y,y[-1]-1)
    if index > 100000:
        break
x=np.delete(x,0)
y=np.delete(y,0)

In [ ]:
import matplotlib.dates as mdates
plt.figure(figsize=(15,7))
plt.plot(x,y,'k-')
plt.gcf().autofmt_xdate()
dtFmt = mdates.DateFormatter('%D-%H:%M') 
plt.gca().xaxis.set_major_formatter(dtFmt) 
plt.xlabel('Time')
plt.ylabel('Change in Bikes')

In [ ]:
from scipy import optimize
def sin_func(x, a, b,c):
    return a * np.sin(x*b*(2*np.pi))+c
xdiff=np.zeros(len(x))
for i in range(len(x)):
    xdiff[i]=(x[i]-x[0]).total_seconds()
par, parcov = optimize.curve_fit(sin_func, xdiff, y,p0=[10,1e-5,-23])
print(par)
print(parcov)

plt.figure(figsize=(15,7))
plt.plot(xdiff,y,'k-')
xdense=np.arange(0,xdiff[-1],100)
plt.plot(xdense,sin_func(xdense,10,1e-5,-23),label='Starting Parameters')
plt.plot(xdense,sin_func(xdense,*par),label='Best-Fit Parameters')

plt.xlabel('Time(s)')
plt.ylabel('Change in Bikes')
plt.legend()

In [ ]:
plt.figure(figsize=(15,7))

from scipy import interpolate
f = interpolate.interp1d(xdiff, y)

time_step = xdense[1]-xdense[0]
yinterp=f(xdense)


freqs = np.fft.rfftfreq(yinterp.size, time_step)
idx = np.argsort(yinterp)
ps = np.abs(np.fft.rfft(yinterp-np.mean(yinterp)))**2

plt.plot(freqs[1:], ps[1:],color='k')
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power ')
plt.axvline(1/86400.,label='1/day')
plt.legend()